In [73]:
from __future__ import annotations
from oop_functions.classifier_oop import *
from oop_functions.util_functions import *
from oop_functions.merge_dataset_functions import *
from oop_functions.analytics_oop import *

from xgboost import XGBClassifier

In [74]:
processed_data = pd.read_csv('./processed_dataset/recent_propagated_dataset.csv', index_col=0)

In [75]:
year = 1
label = f'cancer_in_next_{year}_years'
source_df = resample_class(processed_data, label, 0, 20000)

In [76]:
classifier = XGBClassifier(eval_metric= 'error', learning_rate= 0.1)
experiment_data_helper = ExperimentDataHelper1(source_df, label, ['cancer_'])
data_util_lambdas = experiment_data_helper.train_test_split_util.split_kfold()
analytics_util = AnalyticsUtil(classifier, data_util_lambdas[0]()).fit()